In [4]:
import re 
import numpy as np
import pandas as pd
import glob
import string 
from bs4 import BeautifulSoup,SoupStrainer
from nltk import bigrams
from nltk.collocations import BigramCollocationFinder
from preprocessing import reuters , reuters_parser

In [5]:
reuters , predict_df  , train_df = reuters_parser(reuters)

In [6]:
# before preprocessing
train_df.head()

,description,docID,name,topic,text
0,showers continued throughout the week in the b...,0,bahia cocoa review,cocoa,"[shower, continu, throughout, week, bahia, coc..."
8238,the u s agriculture department reported the fa...,4,national average prices for farmerowned reserve,corn,NaN
8239,the u s agriculture department reported the fa...,4,national average prices for farmerowned reserve,barley,NaN
8240,the u s agriculture department reported the fa...,4,national average prices for farmerowned reserve,oat,NaN
8241,the u s agriculture department reported the fa...,4,national average prices for farmerowned reserve,sorghum,NaN


In [9]:
# Deal with the document with more than one topics
# 
# Original:
# row1:  D1   t1, t2, t3
# 
# New:
# row1:  D1   t1
# row2:  D1   t2
# row3:  D1   t3

def manyTopics(df):

    # Get the number of rows
    rs = df.shape[0]

    for i in range(rs):

        topic = df.iloc[i]['topic']
        
        # Find the row with multiple topics         
        if(' 'in topic):

            # Keep other data the same           
            des = df.iloc[i]['description']
            dID = df.iloc[i]['docID']
            name = df.iloc[i]['name']

            # Split the long string into a list         
            topicList = topic.split()

            for j in topicList:
                
                newOne = {'description':des,
                         'docID':dID,
                         'name':name,
                          'topic':j
                         }
                
                # Append the new row to the end             
                df = df.append(newOne,ignore_index=True)

                # Drop the old row
                df = df.drop(i)

    df = df.sort_values(by=['docID'])  
    
    return df

train_df = manyTopics(train_df)

In [10]:
train_df.fillna(-1,inplace=True)
train_df.head()


,description,docID,name,topic,text
0,showers continued throughout the week in the b...,0,bahia cocoa review,cocoa,"[shower, continu, throughout, week, bahia, coc..."
1,the u s agriculture department reported the fa...,4,national average prices for farmerowned reserve,corn,-1
2,the u s agriculture department reported the fa...,4,national average prices for farmerowned reserve,barley,-1
3,the u s agriculture department reported the fa...,4,national average prices for farmerowned reserve,oat,-1
4,the u s agriculture department reported the fa...,4,national average prices for farmerowned reserve,sorghum,-1


In [11]:
# Feature selection start ====================================
from collections import Counter

# get the total number of rows
train_rows = train_df.shape[0]

allTrainTerms = []

# concatenate all text rows into one list
for i in range(train_rows):
    currTermList = train_df.iloc[i]['text']
    if currTermList != -1 :
        allTrainTerms += currTermList
# sizeAllTerms: 967,103
sizeAllTerms = len(allTrainTerms)


# See the number of unique terms in this list: 68,616
numOfUniques = len(set(allTrainTerms))


# Find the most common word

count = Counter(allTrainTerms)

# return a list of tuples: (term, the number of occurrence in the whole df )
termFreqs = count.most_common(sizeAllTerms)

# the number of terms occurrs more than 100 times: 1,234
morethan100 = 0

for i in termFreqs:
    if i[1] > 100 :
        morethan100 += 1

In [12]:
# choose the popular terms 
# topTerms type: a list of tuples
topTerms = count.most_common(morethan100)
# topTerms[0]:(term:frequencies)
# Feature selection end ====================================

In [9]:
# Insert term occurrence to the dataframe ==================

def count_words(word,index):
    if isinstance(index,int):
        return 0
    return index.count(word)

def insertVS(source,listOfTerms):
    
    df = empty_df.copy()
    
    result = dict(zip(listOfTerms,[[] for i in listOfTerms]))
    
    df = pd.DataFrame(result)
    oc = []
    
    for word in listOfTerms:
        oc.append(source.text.iloc[0].count(word))
    
    cnt = np.vectorize(count_words,excluded=['index'])
    
    occurance = source.text.apply(
        lambda x: np.array(
            cnt(listOfTerms,index=x))).values

    occurance = np.vstack(occurance)

    for (i,col) in enumerate(df.columns):
        df[col] = occurance[:,i]

    df['topic'] =  source.topic.values 
    df['docID'] = source.docID.values
    
    return df
# Insertion End ================================================

In [1]:
knnTrainDf = insertVS(train_df,topTerms)

# knnTrainDf = vector_df.copy()
knnTrainDf.head(10)

NameError: name 'insertVS' is not defined

In [20]:
import math 
from sklearn.neighbors import KNeighborsClassifier 

# Split the feature and ground truth in training data
X = np.array(knnTrainDf.loc[:,(knnTrainDf.columns != 'topic') &
                               (knnTrainDf.columns != 'docID')])

y = np.array(knnTrainDf['topic'])

# Choose k for knn: k = sqrt(n) = 107
k = round(math.sqrt(knnTrainDf.shape[0]))

# Initialize the classifier
classifier = KNeighborsClassifier(n_neighbors = k)

# Fit the model
classifier = classifier.fit(X,y)

# Cleaning the predict data
predict_df = cleaning(predict_df)

NameError: name 'knnTrainDf' is not defined

In [618]:
knnPredict = insertVS(predict_df)

knnPredict.head(10)

,said,mln,dlr,pct,year,ct,wa,net,billion,ha,...,extern,occur,diseas,know,refus,miyazawa,differenti,coordin,topic,docID
0,5,1,2,0,2,0,1,1,0,1,...,0,0,0,0,0,0,0,0,Nan,0
1,0,0,1,0,1,0,3,0,0,0,...,0,0,0,0,0,0,0,0,Nan,1
2,1,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,0,0,0,Nan,2
3,3,0,0,0,2,0,0,0,0,0,...,0,0,1,0,0,0,0,0,Nan,3
4,4,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,Nan,5
5,5,2,6,8,1,0,3,0,0,0,...,0,0,0,0,0,0,0,0,Nan,6
6,3,0,0,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,Nan,7
7,1,1,2,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,Nan,8
8,4,2,3,5,8,0,0,0,1,1,...,0,0,0,0,0,0,0,0,Nan,9
9,3,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,Nan,10


In [620]:
predicted = knnPredict.copy()

def findTopic(lst):
    res = classifier.predict([lst])[0]
    return res


for i in range(predicted.shape[0]):
    features = knnPredict.iloc[i].tolist()[:-2]
    topic = findTopic(features)
    
    predicted.at[i,'topic'] = topic
    
predicted.head()



,said,mln,dlr,pct,year,ct,wa,net,billion,ha,...,extern,occur,diseas,know,refus,miyazawa,differenti,coordin,topic,docID
0,5,1,2,0,2,0,1,1,0,1,...,0,0,0,0,0,0,0,0,acq,0
1,0,0,1,0,1,0,3,0,0,0,...,0,0,0,0,0,0,0,0,acq,1
2,1,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,0,0,0,acq,2
3,3,0,0,0,2,0,0,0,0,0,...,0,0,1,0,0,0,0,0,acq,3
4,4,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,acq,5


In [639]:
# Create a dictionary contains all topic-docID pairs
topicsLst = knnTrainDf.topic.tolist()+predicted.topic.tolist()
docIDsLst = knnTrainDf.docID.tolist()+predicted.docID.tolist()

# Initialize an empty dictionary
topicDict = dict()

for i in range(len(topicsLst)):
    
    k = topicsLst[i]
    v = docIDsLst[i]
    
    # If topic is already saved in the dictionary     
    if(k in topicDict):
        topicDict[k].append(v)
        
    else:
        topicDict[k] = [v]
        
# len(topicDict): 240 topics

240

In [640]:
topicDict

{'interest': [4,
  89,
  130,
  187,
  274,
  408,
  448,
  454,
  457,
  482,
  483,
  655,
  762,
  782,
  821,
  880,
  924,
  1090,
  1155,
  1245,
  1463,
  1502,
  1544,
  2082,
  2199,
  2237,
  2243,
  2262,
  2313,
  2364,
  2421,
  2424,
  2659,
  2715,
  2816,
  2819,
  2830,
  2847,
  2857,
  2860,
  2899,
  2902,
  2906,
  2933,
  2940,
  2959,
  3126,
  3231,
  3234,
  3236,
  3240,
  3276,
  3289,
  3335,
  3343,
  3403,
  3438,
  3835,
  4007,
  4023,
  4104,
  4143,
  4229,
  4230,
  4261,
  4316,
  4320,
  5030,
  5158,
  5309,
  5312,
  5320,
  5459,
  5498,
  5558,
  5563,
  5570,
  5586,
  5598,
  5613,
  5637,
  5651,
  5715,
  5763,
  5768,
  6043,
  6090,
  6175,
  6177,
  6196,
  6349,
  6351,
  6361,
  6363,
  6364,
  6365,
  6368,
  6397,
  6398,
  6425,
  6452,
  6482,
  6530,
  6542,
  6763,
  6921,
  7009,
  7015,
  7018,
  7026,
  7039,
  7072,
  7083,
  7090,
  7100,
  7101,
  7111,
  7124,
  7130,
  7182,
  7228,
  7234,
  7245,
  7250,
  7273,
  7277,


In [641]:
# Module 10b Topic Restriction

def topicFilter(choice):
    docIDs = topicDict.get(choice)
    
    print(docIDs)
topicFilter('soy-oil')

[1174, 3141, 3790, 4172, 6993, 7424, 7785, 9455, 13499, 13750, 13870, 17780]


In [1]:
# Module 9a Automatic Thesaurus Construction
def jaccardSim(term1, term2):
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

str1 = 'AI is our friend and it has been friendly'
str2 = 'AI and humans have always been friendly'

jaccardSim(str1, str2)


0.3333333333333333

In [2]:
empty_df.head()

NameError: name 'empty_df' is not defined

In [ ]:
def single_text_prep(text):

    # Substitute multiple spaces with single white space 
    processed_txt = ' '.join(text.split())

    # Remove the punctuation from the string
    for c in string.punctuation:
        processed_txt = processed_txt.replace(c,"")

    # Remove digits
    processed_txt = re.sub(r"\d+", "", processed_txt)

    # Preprocessing
    processed_txt = sp.preprocessing(processed_txt)

    return processed_txt




In [ ]:
# ===============================================================
# Bigram Language Model
# ===============================================================
# ref: https://bit.ly/2TRJZko 
def bigram_model(list_of_dicts):
    
    # Concatenate all descriptions into one list 
    one_list_word = []
    for i in list_of_dicts:
        curr_txt = i["description"] 
        curr_word_list = single_text_prep(curr_txt)
        one_list_word.append(curr_word_list)

    
    # Flat the list of lists
    one_list_word = [item for sublist in one_list_word for item in sublist]

    finder = BigramCollocationFinder.from_words(one_list_word)
    finder.apply_freq_filter(5)
    bigram_model = finder.ngram_fd.items()
    bigram_model = sorted(finder.ngram_fd.items(), key=lambda item: item[1],reverse=True)  
     
    np.save("bigram_model.npy",bigram_model)



In [1]:
import numpy as np

# load the bigram_model.npy
bigramModel_npy = np.load('bigram_model.npy')

46878

In [3]:
bigramModel_npy[:3]

array([[('reuter', '\x03'), 18856],
       [('mln', 'dlr'), 9109],
       [('billion', 'dlr'), 4166]], dtype=object)

In [5]:
import spider as sp

courses = sp.cources
type(courses)

AttributeError: module 'spider' has no attribute 'cources'

In [ ]:
if __name__ == "__main__":
    flatten = rt_corpus_preprocessing(documents)
    bigram_model(flatten)
    